In [35]:
%matplotlib inline
import numpy as np
from pulp import *
import pandas as pd
import pprint
import csv
MEPS=1.03-6

In [36]:
df=pd.read_csv('premiere.csv',header=0,usecols=[0])
df

,Club
0,Liverpool
1,Manchester City
2,Chelsea
3,Leicester City
4,Manchester United
5,Wolverhampton Wanderers
6,Tottenham Hotspur
7,Sheffield United
8,Burnley
9,Arsenal


In [37]:
DMUs=df.values
print(DMUs)

[['Liverpool']
 ['Manchester City']
 ['Chelsea']
 ['Leicester City']
 ['Manchester United']
 ['Wolverhampton Wanderers']
 ['Tottenham Hotspur']
 ['Sheffield United']
 ['Burnley']
 ['Arsenal']]


In [38]:
dfx=pd.read_csv('premiere.csv',header=0,index_col=0,usecols=[0,3,4,5])
dfx

,Expenditure(€M),Seasons in Premiere League,Club Rating
Club,,,
Liverpool,10.40,28,85
Manchester City,166.82,23,85
Chelsea,45.00,28,81
Leicester City,104.30,14,79
Manchester United,214.00,28,81
Wolverhampton Wanderers,121.80,6,78
Tottenham Hotspur,148.50,28,82
Sheffield United,69.00,4,75
Burnley,19.40,6,76


In [39]:
x=dfx

In [40]:
print(dfx)

                         Expenditure(€M)  Seasons in Premiere League  \
Club                                                                   
Liverpool                          10.40                          28   
Manchester City                   166.82                          23   
Chelsea                            45.00                          28   
Leicester City                    104.30                          14   
Manchester United                 214.00                          28   
Wolverhampton Wanderers           121.80                           6   
Tottenham Hotspur                 148.50                          28   
Sheffield United                   69.00                           4   
Burnley                            19.40                           6   
Arsenal                           160.40                          28   

                         Club Rating  
Club                                  
Liverpool                         85  
Manchester City   

In [41]:
x.shape

(10, 3)

In [42]:
dfy=pd.read_csv('premiere.csv',header=0,usecols=[1,2])
dfy

,Points,Goal Difference
0,93,48
1,75,58
2,63,15
3,62,28
4,62,28
5,59,13
6,58,14
7,54,2
8,54,0
9,53,8


In [43]:
y=dfy

In [44]:
y.shape

(10, 2)

In [45]:
n,m=x.shape
n,s=y.shape

In [46]:
res=[]

In [47]:
for o in range(n):
    prob=LpProblem('DMU_'+str(o),LpMaximize)
    #　重み
    v=[LpVariable('v'+str(i),lowBound=0,
                  cat='Continuous') for i in range(m)]
    u=[LpVariable('u'+str(i),lowBound=0,
                  cat='Continuous') for i in range(s)]
    #　目的関数
    prob += lpDot(u,y.iloc[o,:])

    # 制約条件
    prob += lpDot(v,x.iloc[o,:])==1,'Normalize'+str(o)

    for j in range(n):
        prob += lpDot(u,y.iloc[j,:]) <= lpDot(v,x.iloc[j,:])

    prob.solve()
    vs=np.array([v[i].varValue for i in range(m)]) #v*
    us=np.array([u[i].varValue for i in range(s)]) #u*
    print('DMU_'+str(o),': ',DMUs[o])
    print('vs: ',vs)
    print('us: ',us)
    print(' ')
#     # 参照集合作成
#     (eo,)=np.where(np.abs(np.dot(vs,x)-np.dot(us,y))<= MEPS)

#     res.append([DMUs[o],value(prob.objective),
#                 set(eo),tuple(vs),tuple(us)])

    res.append([DMUs[o],value(prob.objective)])

DMU_0 :  ['Liverpool']
vs:  [0.         0.         0.01176471]
us:  [0.01075269 0.        ]
 
DMU_1 :  ['Manchester City']
vs:  [0.00202302 0.02880525 0.        ]
us:  [0.         0.01724138]
 
DMU_2 :  ['Chelsea']
vs:  [0.         0.         0.01234568]
us:  [0.01128368 0.        ]
 
DMU_3 :  ['Leicester City']
vs:  [0.00236831 0.05378466 0.        ]
us:  [0.00581532 0.02062035]
 
DMU_4 :  ['Manchester United']
vs:  [0.         0.         0.01234568]
us:  [0.01128368 0.        ]
 
DMU_5 :  ['Wolverhampton Wanderers']
vs:  [0.00010612 0.02327463 0.01086445]
us:  [0.01694915 0.        ]
 
DMU_6 :  ['Tottenham Hotspur']
vs:  [0.         0.         0.01219512]
us:  [0.01114608 0.        ]
 
DMU_7 :  ['Sheffield United']
vs:  [0.00011595 0.02542969 0.01187041]
us:  [0.01851852 0.        ]
 
DMU_8 :  ['Burnley']
vs:  [0.00138509 0.02909649 0.01050724]
us:  [0.01851852 0.        ]
 
DMU_9 :  ['Arsenal']
vs:  [0.     0.     0.0125]
us:  [0.01142473 0.        ]
 


In [48]:
print(res)

[[array(['Liverpool'], dtype=object), 0.999999984], [array(['Manchester City'], dtype=object), 0.9999999820000001], [array(['Chelsea'], dtype=object), 0.710872155], [array(['Leicester City'], dtype=object), 0.9379192644000001], [array(['Manchester United'], dtype=object), 0.69958847], [array(['Wolverhampton Wanderers'], dtype=object), 1.000000027], [array(['Tottenham Hotspur'], dtype=object), 0.646472582], [array(['Sheffield United'], dtype=object), 1.000000026], [array(['Burnley'], dtype=object), 1.000000026], [array(['Arsenal'], dtype=object), 0.6055107430000001]]


In [49]:
df_f=pd.DataFrame(res,columns=['国名','効率値'])
df_s=df_f.sort_values('効率値',ascending=False)
df_s

,国名,効率値
5,[Wolverhampton Wanderers],1.000000
7,[Sheffield United],1.000000
8,[Burnley],1.000000
0,[Liverpool],1.000000
1,[Manchester City],1.000000
3,[Leicester City],0.937919
2,[Chelsea],0.710872
4,[Manchester United],0.699588
6,[Tottenham Hotspur],0.646473
9,[Arsenal],0.605511


In [71]:
# def DEA_CCR(x,y,DMUs):
#     n,m=x.shape
#     n,s=y.shape
    
#     res=[]
    
#     for o in range(n):
#         prob=LpProblem('DMU_'+str(o),LpMaximize)
#         #　重み
#         v=[LpVariable('v'+str(i),lowBound=0,
#                       cat='Continuous') for i in range(m)]
#         u=[LpVariable('u'+str(i),lowBound=0,
#                       cat='Continuous') for i in range(s)]
#         #　目的関数
#         prob += lpDot(u,y.iloc[:,o])

#         # 制約条件
#         prob += lpDot(v,x.iloc[:,o])==1,'Normalize'+str(o)

#         for j in range(n):
#             prob += lpDot(u,y.iloc[:,j]) <= lpDot(v,x.iloc[:,j])

#         prob.solve()
#         vs=np.array([v[i].varValue for i in range(m)]) #v*
#         us=np.array([v[i].varValue for i in range(m)]) #u*
#         # 参照集合作成
#         (eo,)=np.where(np.abs(np.dot(vs,x)-np.dot(us,y))<= MEPS)
#         res.append([DMUs[o],value(prob.objective),
#                     set(eo),tuple(vs),tuple(us)])
#     return res